In [28]:
import sqlite3
from Bio import Entrez
from Bio import SeqIO
conn = sqlite3.connect("my.db")
c = conn.cursor()

In [53]:

enzyme = ['EC:5.4.2.2', 'EC:5.1.3.3', 'EC:2.3.1.12', 'EC:1.2.4.1', 'EC:2.7.6.1', 'EC:2.2.1.1', 'EC:2.2.1.2','EC:4.1.2.13', 'EC:4.2.1.2', 'EC:2.3.1.61','EC:1.8.1.4','EC:1.2.4.2']
orgnism = ['Homo sapiens', 'Drosophila melanogaster', 'Escherichia coli']

gene_annotation = []
gene_name = []
gene_id = []
list_gene = []
for e in enzyme:
    for o in orgnism:
        Entrez.email = "yixun_tan@berkeley.edu"
        handle = Entrez.esearch(db='gene',
                                retmax = 1,
                                term= e + ' ' + o,
                                sort='relevance',
                                idtype='acc')
        for i in Entrez.read(handle)['IdList']:
            gene_data = Entrez.efetch(db='gene',id=i,retmode='text')
           # print(gene_data.read())
            list_gene = gene_data.readlines()
            for x in list_gene:
                if "Annotation" in x:
                    gene_annotation.append(x.strip("Annotation:\n"))
                if "ID" in x:
                    gene_id.append(int(x.strip("ID:\n").lstrip()))
            gene_name.append(list_gene[2].strip("\n"))         
#print(gene_name)
#print(gene_annotation)
#print(gene_id)

### Using ENTREZ package to search and fetch data from online database.
Acquire gene name, description and locus in chromosome from **Gene** database, using the basic summary type.
Then, generate 3 lists which contain information about name, annotation, and id:

*gene_name*

*gene_annotaion*

*gene_id*

In [54]:
chromosome = []

for string in gene_annotation:
    word1 = ''
    for i in string:
        if i == '(':
            chromosome.append(word1.rstrip().lstrip())
        else:
            word1 = word1 + i

### Extract chromosome information from gene_annotation.
Generate another list that contains the information about where those genes locate.

*chromosome*

In [55]:
start = []
start_end = []
end = []
for i in range(len(gene_annotation)):    
    word1 = gene_annotation[i].replace(chromosome[i],"").lstrip().strip("()complement").rstrip().rstrip(",")
    start_end.append(word1)
    word2 = ""
    for i in word1:
        if i == ".":
            start.append(word2)
            break
        else:
            word2 += i
for i in range(len(start)):
    end.append(start_end[i].replace(start[i],"").strip("."))

### Extract start and end position from gene_annotation.
Generate another 2 list that contains the information about where those genes specifically locate on chromosome.

*start*

*end*

In [27]:
for i in range(36): # Change the data type of start and end from string to int.
    start[i] = int(start[i])
    end[i] = int(end[i])

In [23]:
c.execute("""DROP TABLE genes;""")
c.execute("""CREATE TABLE genes (id INT PRIMARY KEY ASC,
                                 name_description TEXT,
                                 chromosome TEXT,
                                 start INT,
                                 end INT,
                                 strand VARCHAR(1));""")
for i in range(36):
    c.execute("""INSERT INTO genes (id,
                                    name_description,
                                    chromosome,
                                    start,
                                    end,
                                    strand)
                             VALUES (?,
                                     ?,
                                     ?,
                                     ?,
                                     ?,
                                     '-');""",(gene_id[i],gene_name[i],chromosome[i],start[i],end[i]))
conn.commit()


In [31]:
from beautifultable import BeautifulTable as BT
c.execute("""SELECT * FROM genes ORDER BY id;""")
table = c.fetchall()
beautiful_table_gene = BT()
beautiful_table_gene.column_headers = ["ID", "Name & Description", "Chromosome", "Start", "End","Strand"]
for row in table:
    beautiful_table_gene.append_row(row)
beautiful_table_gene.WEP_WRAP
print(beautiful_table_gene)


+----------+-----------------------------+----+-----------+-----------+--------+
|    ID    |     Name & Description      | Ch |   Start   |    End    | Strand |
|          |                             | ro |           |           |        |
|          |                             | mo |           |           |        |
|          |                             | so |           |           |        |
|          |                             | me |           |           |        |
+----------+-----------------------------+----+-----------+-----------+--------+
|   226    | Official Symbol: ALDOA and  | Ch | 30053090  | 30070420  |   -    |
|          | Name: aldolase, fructose-bi | ro |           |           |        |
|          | sphosphate A [Homo sapiens  | mo |           |           |        |
|          |          (human)]           | so |           |           |        |
|          |                             | me |           |           |        |
|          |                

### Create a "gene" table and insert data.
Based on what I have searched and fetched online, create a table called "**gene**", including its ID, name and description, chromosome, start and end information. The result is printed as above.

In [58]:
pathways_name = ["Glycolysis", "Citric acid cycle", "Pentose phosphate"]
pathways_description = ["The breakdown of glucose by enzymes, releasing energy and pyruvic acid.", "A series of chemical reactions used by all aerobic organisms to release stored energy.", "A metabolic pathway parallel to glycolysis."]
c.execute("""DROP TABLE pathways;""")
c.execute("""CREATE TABLE pathways (name TEXT PRIMARY KEY,
                                        description TEXT);""")
for i in range(3):   
    c.execute("""INSERT INTO pathways(name,
                                      description)
                             VALUES(?,?)""",(pathways_name[i], pathways_description[i]))
conn.commit()

In [51]:
c.execute("""SELECT * FROM pathways;""")
table = c.fetchall()
beautiful_table_pathway = BT()
beautiful_table_pathway.column_headers = ["Name", "Description"]
for row in table:
    beautiful_table_pathway.append_row(row)
beautiful_table_pathway.WEP_WRAP
print(beautiful_table_pathway)

+-------------------+----------------------------------------------------------+
|       Name        |                       Description                        |
+-------------------+----------------------------------------------------------+
|    Glycolysis     | The breakdown of glucose by enzymes, releasing energy an |
|                   |                     d pyruvic acid.                      |
+-------------------+----------------------------------------------------------+
| Citric acid cycle | A series of chemical reactions used by all aerobic organ |
|                   |              isms to release stored energy.              |
+-------------------+----------------------------------------------------------+
| Pentose phosphate |       A metabolic pathway parallel to glycolysis.        |
+-------------------+----------------------------------------------------------+


### Create a "pathway" table and insert data.
Based on what I have searched and fetched online, create a table called "**pathway**", including its name and description. The result is printed as above.

In [59]:
c.execute("""DROP TABLE associative;""")
c.execute("""CREATE TABLE associative (Gene_id int,
                                        Pathway TEXT);""")
for i in range(len(gene_id)):
    for j in range(3):
        if i // 12 == j:
            c.execute("""INSERT INTO associative(Gene_id,
                                                 Pathway)
                                             VALUES(?,?)""",(gene_id[i], pathways_name[j]))
conn.commit()

In [52]:
c.execute("""SELECT * FROM associative;""")
table = c.fetchall()
beautiful_table_associative = BT()
beautiful_table_associative.column_headers = ["Gene ID", "Pathway"]
for row in table:
    beautiful_table_associative.append_row(row)
beautiful_table_associative.WEP_WRAP
print(beautiful_table_associative)

+----------+-------------------+
| Gene ID  |      Pathway      |
+----------+-------------------+
|   5236   |    Glycolysis     |
+----------+-------------------+
|  44010   |    Glycolysis     |
+----------+-------------------+
|  945271  |    Glycolysis     |
+----------+-------------------+
|  130589  |    Glycolysis     |
+----------+-------------------+
|  40406   |    Glycolysis     |
+----------+-------------------+
|  944943  |    Glycolysis     |
+----------+-------------------+
|   1737   |    Glycolysis     |
+----------+-------------------+
|  34021   |    Glycolysis     |
+----------+-------------------+
|  944794  |    Glycolysis     |
+----------+-------------------+
|   5160   |    Glycolysis     |
+----------+-------------------+
|  31406   |    Glycolysis     |
+----------+-------------------+
|  944834  |    Glycolysis     |
+----------+-------------------+
|   5631   | Citric acid cycle |
+----------+-------------------+
|  43637   | Citric acid cycle |
+---------

### Create a "associate" table and insert data.
Based on what I have searched and fetched online, create a table called "**associate**", including all the gene id and what pathway it belongs to. The result is printed as above.